# Model recovery attack in split learning with multiple data owners

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import mra
from mra import *

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

df = pd.read_excel('../datasets/credit-card.xls', header=1, index_col=0).sample(frac=1)
min_values = df.drop(columns=["default payment next month"]).describe().transpose()['min'].to_numpy()
max_values = df.drop(columns=["default payment next month"]).describe().transpose()['max'].to_numpy()
x = df.drop(columns=["default payment next month"]).to_numpy()
x = (x-min_values)/(max_values-min_values)
y = df["default payment next month"].to_numpy().reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

2022-02-10 16:52:55.539222: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-10 16:53:03.411657: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22336 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:3b:00.0, compute capability: 8.6
2022-02-10 16:53:03.414509: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 3164 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:5e:00.0, compute capability: 8.6
2022-02-10 16:53:03.418803: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:

In [3]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.BatchNormalization()(xin)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(128, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(256, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.Model(xin, output)

In [4]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.binary_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=500
    )

2022-02-10 16:53:17.031678: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Iteration 0001: Training loss: 0.7349
Iteration 0501: Training loss: 0.3287
Iteration 1001: Training loss: 0.6484
Iteration 1501: Training loss: 0.4484
Iteration 2001: Training loss: 0.4445
Iteration 2501: Training loss: 0.3745
Iteration 3001: Training loss: 0.7712
Iteration 3501: Training loss: 0.4008
Iteration 4001: Training loss: 0.3526
Iteration 4501: Training loss: 0.3465


In [5]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(23, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [6]:
log_attack = mra.attack(
    attack_iter=10,
    make_generator=make_generator,
    input_noise=True,
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.2267, RG-normal: 0.2070, reconstruction validation: 0.1034
Iteration 0002: RG-uniform: 0.2277, RG-normal: 0.2070, reconstruction validation: 0.1065
Iteration 0003: RG-uniform: 0.2090, RG-normal: 0.1967, reconstruction validation: 0.1141


KeyboardInterrupt: 